<!-- # Copyright (c) 2024 Graphcore Ltd. All rights reserved. -->

# Collect value statistics for formats

This notebook presents various statistics for a variety of float formats.
Some of these are present on the `FormatInfo` class, and are presented for all formats.
Others are obtained by enumerating all values (only for the =16 bit formats). 

## Statistics obtained from FormatInfo

 - name: Format
 - B: Bits in the format
 - P: Precision in bits
 - E: Exponent field width in bits
 - T: Trailing significand field width in bits
 - max: Largest finite value
 - min: Most negative value (typically the same, unless twos complement)
 - smallest: Smallest positive value
 - smallest_normal: Smallest positive normal value, NaN if all finite values are subnormal
 

In [1]:
%run utils.py
D = pandas_render  # from utils
import pandas
from functools import partial
from gfloat import *
from gfloat.formats import *

import numpy as np


# Special rendering for float values -
# if they don't round-trip in 8.5g, prepend with "≈", or render as rational
def render_float(approx: bool, v):
    if not isinstance(v, float):
        return str(v)

    if np.isnan(v):
        return "n/a"

    s = f"{v:8.5g}"
    if float(s) == v:
        return s

    if approx:
        return "≈" + s
    else:
        return float_pow2str(v)


def collect_stats(fi: FormatInfo):
    return dict(
        name=fi.name,
        B=fi.bits,
        P=fi.precision,
        E=fi.expBits,
        smallest=fi.smallest,
        smallest_normal=fi.smallest_normal if not fi.is_all_subnormal else np.nan,
        max=fi.max,
        num_nans=float(fi.num_nans),
        infs=fi.num_infs,
    )


stats = [collect_stats(fi) for fi in sample_formats]
df = pandas.DataFrame(stats)
D(df, format=partial(render_float, True))


## Statistics computed by exhaustive inspection

 - lt1: Number of values x such that `0 < x < 1`
 - gt1: Number of values x such that `1 < x < Inf`
 - rt16: True if all values are exactly representable in IEEE binary16
 - min/maxSubnormal: Smallest/largest subnormal value, "n/a" if no values are subnormal
 - min/maxNormal: Smallest/largest normal value, "n/a" if no values are normal


In [2]:
def compute_stats(fi: FormatInfo):
    # Generate all values
    values = [decode_float(fi, i) for i in range(2**fi.bits)]
    df = pandas.DataFrame(values)

    # Compute statistics: lt1,gt1
    fval = df["fval"]
    total_01 = fval.between(0, 1, inclusive="neither").sum()
    total_1Inf = fval.between(1, np.inf, inclusive="neither").sum()

    # Compute statistics: maxFinite,minFinite
    finite_vals = fval[np.isfinite(fval)]
    maxFinite = finite_vals.loc[finite_vals.idxmax()]
    minFinite = finite_vals.loc[finite_vals.idxmin()]
    assert maxFinite == fi.max
    assert minFinite == fi.min
    fi.emax

    # Compute statistics: maxNormal,minNormal
    normal_vals = fval[(df["fclass"] == FloatClass.NORMAL) & (fval > 0)]
    maxNormal = normal_vals.loc[normal_vals.idxmax()] if normal_vals.any() else np.nan
    minNormal = normal_vals.loc[normal_vals.idxmin()] if normal_vals.any() else np.nan
    assert np.isnan(maxNormal) or maxNormal == fi.max
    assert np.isnan(minNormal) or minNormal == fi.smallest_normal

    # Compute statistics: minSubnormal
    pos_subnormal = fval[(df["fclass"] == FloatClass.SUBNORMAL) & (fval > 0)]
    maxSubnormal = (
        pos_subnormal.loc[pos_subnormal.idxmax()] if pos_subnormal.any() else np.nan
    )
    minSubnormal = (
        pos_subnormal.loc[pos_subnormal.idxmin()] if pos_subnormal.any() else np.nan
    )
    assert np.isnan(minSubnormal) or minSubnormal == fi.smallest_subnormal
    assert np.isnan(minNormal) or minNormal == fi.smallest_normal

    assert np.nanmin([minSubnormal, minNormal]) == fi.smallest

    # Compute roundtrips: rt16, rt32
    with np.errstate(over="ignore"):
        rt16 = (np.float64(np.float16(fval)) == np.float64(fval)) | ~np.isfinite(fval)
        rt32 = (np.float64(np.float32(fval)) == np.float64(fval)) | ~np.isfinite(fval)

    rt16 = rt16.all()
    rt32 = rt32.all()
    # assert rt32  # If not, we should include rt32 in the table

    # Assemble tuple
    return dict(
        name=fi.name,
        B=fi.bits,
        P=fi.precision,
        E=fi.expBits,
        rt16=rt16,
        rt32=rt32,
        lt1=total_01,
        gt1=total_1Inf,
        minSubnormal=minSubnormal,
        maxSubnormal=maxSubnormal,
        minNormal=minNormal,
        maxNormal=maxNormal,
    )


stats = [compute_stats(fi) for fi in sample_formats if fi.bits <= 16]
df2 = pandas.DataFrame(stats)
D(df2, format=partial(render_float, True))

### Emit the same table, but with exact values

In this table, float values are printed as decimals, unless the decimals are not an
exact representation of the value, in which case, they are printed as rationals (between 1 and 2) times 2^E.

In [3]:
D(df2, format=partial(render_float, False))

In [4]:
count_p3109_formats = len(
    [
        1
        for signedness in (True, False)
        for domain in (Domain.Extended, Domain.Finite)
        for k in range(2, 16)
        for p in range(1, k if signedness else k + 1)
    ]
)
count_p3109_formats, count_p3109_formats == format_info_ocp_e4m3.max

(448, True)

In [5]:
all_p3109 = [
    format_info_p3109(k, p, signedness, domain)
    for signedness in (Signedness.Signed, Signedness.Unsigned)
    for domain in (Domain.Extended, Domain.Finite)
    for k in range(3, 8)
    for p in range(1, k if signedness == Signedness.Signed else k + 1)
]

stats = [compute_stats(fi) for fi in all_p3109]
df3 = pandas.DataFrame(stats)
df3["rt32"].all()
# D(df3, format=partial(render_float, True))

np.True_

## Tables in RST/Markdown

These are used to generate gfloat documentation, but may be of use in other
contexts so left here.

In [6]:
from tabulate import tabulate

dfstr = df.map(lambda x: render_float(True, x))
print(
    tabulate(dfstr, df.columns, tablefmt="rst", showindex=False).replace(" nan", " n/a")
)

============  ===  ===  ===  ===========  =================  ============  ===========  ======
name            B    P    E  smallest     smallest_normal    max           num_nans       infs
============  ===  ===  ===  ===========  =================  ============  ===========  ======
p3109_k3p2sf    3    2    1  0.5          1                  1.5           1                 0
ocp_e2m1        4    2    2  0.5          1                  6             0                 0
p3109_k4p2sf    4    2    2  0.25         0.5                3             1                 0
ocp_e2m3        6    4    2  0.125        1                  7.5           0                 0
ocp_e3m2        6    3    3  0.0625       0.25               28            0                 0
p3109_k6p3sf    6    3    3  0.03125      0.125              14            1                 0
p3109_k6p4sf    6    4    2  0.0625       0.5                3.75          1                 0
ocp_e4m3        8    4    4  ≈0.0019531   0.015625

In [7]:
from tabulate import tabulate

dfstr = df.map(lambda x: render_float(False, x))
print(tabulate(dfstr, df.columns, tablefmt="rst", showindex=False))

============  ===  ===  ===  ===========  =================  ========================================  ======================================  ======
name            B    P    E  smallest     smallest_normal    max                                       num_nans                                  infs
============  ===  ===  ===  ===========  =================  ========================================  ======================================  ======
p3109_k3p2sf    3    2    1  0.5          1                  1.5                                       1                                            0
ocp_e2m1        4    2    2  0.5          1                  6                                         0                                            0
p3109_k4p2sf    4    2    2  0.25         0.5                3                                         1                                            0
ocp_e2m3        6    4    2  0.125        1                  7.5                                    

In [8]:
dfstr = df2.map(lambda x: render_float(False, x))
print(
    tabulate(dfstr, df2.columns, tablefmt="github", showindex=False).replace(
        " nan", " n/a"
    )
)

| name         |   B |   P |   E | rt16   | rt32   |   lt1 |   gt1 | minSubnormal   | maxSubnormal   | minNormal   | maxNormal     |
|--------------|-----|-----|-----|--------|--------|-------|-------|----------------|----------------|-------------|---------------|
| p3109_k3p2sf |   3 |   2 |   1 | True   | True   |     1 |     1 | 0.5            | 0.5            | 1           | 1.5           |
| ocp_e2m1     |   4 |   2 |   2 | True   | True   |     1 |     5 | 0.5            | 0.5            | 1           | 6             |
| p3109_k4p2sf |   4 |   2 |   2 | True   | True   |     3 |     3 | 0.25           | 0.25           | 0.5         | 3             |
| ocp_e2m3     |   6 |   4 |   2 | True   | True   |     7 |    23 | 0.125          | 0.875          | 1           | 7.5           |
| ocp_e3m2     |   6 |   3 |   3 | True   | True   |    11 |    19 | 0.0625         | 0.1875         | 0.25        | 28            |
| p3109_k6p3sf |   6 |   3 |   3 | True   | True   |    15 |    15 | 